# EEG Markers of Affect

This notebook demonstrates extraction and analysis of EEG markers related to emotional states, particularly frontal alpha asymmetry.

## Background

### Frontal Alpha Asymmetry
- **Left frontal activity**: Approach motivation, positive affect
- **Right frontal activity**: Withdrawal motivation, negative affect
- **Asymmetry index**: Left - Right alpha power (or log ratio)

### Applications in BrainJam
- Real-time mood monitoring
- Correlating affect with music generation
- Validating emotional impact of interventions
- Neurofeedback for mood regulation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import signal, stats
from scipy.signal import welch, butter, filtfilt
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Set random seed
np.random.seed(42)

print("Libraries loaded successfully")

## 1. Simulate EEG Data

We'll simulate EEG data from frontal channels (F3, F4) with different asymmetry patterns.

In [ ]:
def simulate_eeg_with_asymmetry(duration_sec, fs=250, asymmetry_direction='positive'):
    """
    Simulate EEG data with frontal alpha asymmetry
    
    Parameters:
    -----------
    duration_sec: float
        Duration in seconds
    fs: int
        Sampling frequency in Hz
    asymmetry_direction: str
        'positive' (left > right, approach/positive affect)
        'negative' (right > left, withdrawal/negative affect)
        'neutral' (no asymmetry)
    
    Returns:
    --------
    dict with 'F3' and 'F4' channel data
    """
    n_samples = int(duration_sec * fs)
    t = np.arange(n_samples) / fs
    
    # Base alpha oscillation (8-13 Hz)
    alpha_freq = 10  # Hz
    
    # Create alpha activity with different amplitudes based on asymmetry
    if asymmetry_direction == 'positive':
        # More left alpha (less left activity = more positive affect)
        # Note: Alpha power is inversely related to cortical activity
        f3_alpha_amp = 3.0  # Higher alpha = lower activity
        f4_alpha_amp = 2.0  # Lower alpha = higher activity
    elif asymmetry_direction == 'negative':
        # More right alpha (less right activity = more negative affect)
        f3_alpha_amp = 2.0
        f4_alpha_amp = 3.0
    else:  # neutral
        f3_alpha_amp = 2.5
        f4_alpha_amp = 2.5
    
    # Generate signals
    f3 = f3_alpha_amp * np.sin(2 * np.pi * alpha_freq * t + np.random.randn())
    f4 = f4_alpha_amp * np.sin(2 * np.pi * alpha_freq * t + np.random.randn())
    
    # Add other frequency components
    # Theta (4-7 Hz)
    theta_freq = 6
    f3 += 1.5 * np.sin(2 * np.pi * theta_freq * t)
    f4 += 1.5 * np.sin(2 * np.pi * theta_freq * t)
    
    # Beta (13-30 Hz)
    beta_freq = 20
    f3 += 0.8 * np.sin(2 * np.pi * beta_freq * t)
    f4 += 0.8 * np.sin(2 * np.pi * beta_freq * t)
    
    # Add noise
    f3 += np.random.randn(n_samples) * 0.5
    f4 += np.random.randn(n_samples) * 0.5
    
    return {'F3': f3, 'F4': f4, 'time': t, 'fs': fs}

# Simulate three conditions
duration = 60  # 60 seconds
positive_data = simulate_eeg_with_asymmetry(duration, asymmetry_direction='positive')
negative_data = simulate_eeg_with_asymmetry(duration, asymmetry_direction='negative')
neutral_data = simulate_eeg_with_asymmetry(duration, asymmetry_direction='neutral')

print(f"Simulated {duration}s of EEG data for 3 conditions")
print(f"Sampling rate: {positive_data['fs']} Hz")
print(f"Number of samples: {len(positive_data['F3'])}")

## 2. Visualize Raw Data

In [ ]:
# Plot first 5 seconds of each condition
fig, axes = plt.subplots(3, 1, figsize=(14, 10))
plot_duration = 5  # seconds
n_plot = int(plot_duration * positive_data['fs'])

conditions = [
    (positive_data, 'Positive Affect (Approach)', 'green'),
    (neutral_data, 'Neutral', 'gray'),
    (negative_data, 'Negative Affect (Withdrawal)', 'red')
]

for ax, (data, title, color) in zip(axes, conditions):
    ax.plot(data['time'][:n_plot], data['F3'][:n_plot], label='F3 (Left)', alpha=0.7)
    ax.plot(data['time'][:n_plot], data['F4'][:n_plot], label='F4 (Right)', alpha=0.7)
    ax.set_xlabel('Time (s)', fontsize=11)
    ax.set_ylabel('Amplitude (μV)', fontsize=11)
    ax.set_title(title, fontsize=13, fontweight='bold')
    ax.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Extract Alpha Band Power

In [ ]:
def compute_band_power(signal_data, fs, freq_band):
    """
    Compute power in specific frequency band using Welch's method
    
    Parameters:
    -----------
    signal_data: array
        EEG signal
    fs: int
        Sampling frequency
    freq_band: tuple
        (low_freq, high_freq) in Hz
    
    Returns:
    --------
    float: Power in the frequency band
    """
    # Compute power spectral density
    freqs, psd = welch(signal_data, fs=fs, nperseg=2*fs)
    
    # Find indices for frequency band
    idx_band = np.logical_and(freqs >= freq_band[0], freqs <= freq_band[1])
    
    # Integrate power in band
    band_power = np.trapz(psd[idx_band], freqs[idx_band])
    
    return band_power, freqs, psd

# Define alpha band
alpha_band = (8, 13)  # Hz

# Compute alpha power for each condition
results = {}
for name, data in [('Positive', positive_data), 
                   ('Neutral', neutral_data), 
                   ('Negative', negative_data)]:
    f3_power, freqs, f3_psd = compute_band_power(data['F3'], data['fs'], alpha_band)
    f4_power, _, f4_psd = compute_band_power(data['F4'], data['fs'], alpha_band)
    
    results[name] = {
        'F3_alpha': f3_power,
        'F4_alpha': f4_power,
        'freqs': freqs,
        'F3_psd': f3_psd,
        'F4_psd': f4_psd
    }
    
    print(f"\n{name} Condition:")
    print(f"  F3 alpha power: {f3_power:.3f}")
    print(f"  F4 alpha power: {f4_power:.3f}")

## 4. Visualize Power Spectral Density

In [ ]:
# Plot PSD for all conditions
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

condition_names = ['Positive', 'Neutral', 'Negative']
colors = ['green', 'gray', 'red']

for ax, cond, color in zip(axes, condition_names, colors):
    res = results[cond]
    
    # Plot PSD
    ax.semilogy(res['freqs'], res['F3_psd'], label='F3 (Left)', linewidth=2, alpha=0.7)
    ax.semilogy(res['freqs'], res['F4_psd'], label='F4 (Right)', linewidth=2, alpha=0.7)
    
    # Highlight alpha band
    ax.axvspan(alpha_band[0], alpha_band[1], alpha=0.2, color=color, label='Alpha band')
    
    ax.set_xlim(1, 40)
    ax.set_xlabel('Frequency (Hz)', fontsize=11)
    ax.set_ylabel('Power Spectral Density (μV²/Hz)', fontsize=11)
    ax.set_title(f'{cond} Affect', fontsize=13, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Calculate Frontal Alpha Asymmetry

In [ ]:
def calculate_asymmetry(left_power, right_power, method='difference'):
    """
    Calculate frontal asymmetry index
    
    Note: Higher alpha = lower cortical activity (inverse relationship)
    
    Parameters:
    -----------
    left_power: float
        Alpha power from left electrode (F3)
    right_power: float
        Alpha power from right electrode (F4)
    method: str
        'difference': right - left (common in literature)
        'log_ratio': ln(right) - ln(left)
    
    Returns:
    --------
    float: Asymmetry index
        Positive: Relatively more left activity (approach, positive affect)
        Negative: Relatively more right activity (withdrawal, negative affect)
    """
    if method == 'difference':
        # Right - Left (more common)
        # Since alpha is inversely related to activity:
        # Positive asymmetry = higher right alpha = lower right activity = more left activity
        asymmetry = right_power - left_power
    elif method == 'log_ratio':
        asymmetry = np.log(right_power) - np.log(left_power)
    
    return asymmetry

# Calculate asymmetry for each condition
asymmetry_scores = {}
for name in condition_names:
    asym = calculate_asymmetry(results[name]['F3_alpha'], 
                               results[name]['F4_alpha'], 
                               method='difference')
    asymmetry_scores[name] = asym
    
    print(f"{name} Asymmetry: {asym:.3f}")
    if asym > 0:
        print("  → Approach motivation / Positive affect")
    elif asym < 0:
        print("  → Withdrawal motivation / Negative affect")
    else:
        print("  → Neutral")

## 6. Visualize Asymmetry Scores

In [ ]:
# Bar plot of asymmetry scores
fig, ax = plt.subplots(figsize=(10, 6))

conditions = list(asymmetry_scores.keys())
scores = list(asymmetry_scores.values())
colors_map = {'Positive': 'green', 'Neutral': 'gray', 'Negative': 'red'}
bar_colors = [colors_map[c] for c in conditions]

bars = ax.bar(conditions, scores, color=bar_colors, alpha=0.7, edgecolor='black', linewidth=2)
ax.axhline(y=0, color='black', linestyle='--', linewidth=2, alpha=0.5)
ax.set_ylabel('Asymmetry Index (R-L Alpha Power)', fontsize=12)
ax.set_xlabel('Condition', fontsize=12)
ax.set_title('Frontal Alpha Asymmetry Across Conditions', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Add labels
ax.text(0.02, 0.95, 'Approach/Positive →', transform=ax.transAxes, 
        fontsize=10, verticalalignment='top', color='green', fontweight='bold')
ax.text(0.02, 0.05, '← Withdrawal/Negative', transform=ax.transAxes,
        fontsize=10, verticalalignment='bottom', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

## 7. Multi-Participant Simulation

Simulate data from multiple participants to demonstrate group analysis.

In [ ]:
# Simulate 30 participants, pre and post music intervention
n_participants = 30
duration_short = 30  # seconds per recording

pre_asymmetry = []
post_asymmetry = []

for i in range(n_participants):
    # Pre: More neutral to slightly negative
    pre_data = simulate_eeg_with_asymmetry(duration_short, asymmetry_direction='neutral')
    f3_pre, _, _ = compute_band_power(pre_data['F3'], pre_data['fs'], alpha_band)
    f4_pre, _, _ = compute_band_power(pre_data['F4'], pre_data['fs'], alpha_band)
    asym_pre = calculate_asymmetry(f3_pre, f4_pre)
    asym_pre += np.random.normal(-0.2, 0.5)  # Add individual variability
    pre_asymmetry.append(asym_pre)
    
    # Post: More positive (intervention effect)
    post_data = simulate_eeg_with_asymmetry(duration_short, asymmetry_direction='positive')
    f3_post, _, _ = compute_band_power(post_data['F3'], post_data['fs'], alpha_band)
    f4_post, _, _ = compute_band_power(post_data['F4'], post_data['fs'], alpha_band)
    asym_post = calculate_asymmetry(f3_post, f4_post)
    asym_post += np.random.normal(0.3, 0.4)  # Shift toward positive
    post_asymmetry.append(asym_post)

pre_asymmetry = np.array(pre_asymmetry)
post_asymmetry = np.array(post_asymmetry)

print(f"Simulated {n_participants} participants")
print(f"Pre-intervention asymmetry: M = {pre_asymmetry.mean():.3f}, SD = {pre_asymmetry.std():.3f}")
print(f"Post-intervention asymmetry: M = {post_asymmetry.mean():.3f}, SD = {post_asymmetry.std():.3f}")

## 8. Statistical Analysis

In [ ]:
# Paired t-test
t_stat, p_value = stats.ttest_rel(post_asymmetry, pre_asymmetry)

# Effect size (Cohen's d for paired samples)
diff = post_asymmetry - pre_asymmetry
cohens_d = diff.mean() / diff.std()

print("="*60)
print("STATISTICAL ANALYSIS: Pre-Post Music Intervention")
print("="*60)
print(f"\nPaired t-test: t({n_participants-1}) = {t_stat:.3f}, p = {p_value:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")

if p_value < 0.05:
    print("\n✓ Significant increase in approach-related asymmetry")
    print("  Interpretation: Music intervention enhanced positive affect")
else:
    print("\n✗ No significant change in asymmetry")

print("\n" + "="*60)

In [ ]:
# Visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Pre vs Post scatter
axes[0].scatter(pre_asymmetry, post_asymmetry, alpha=0.6, s=80, color='purple')
axes[0].plot([pre_asymmetry.min(), pre_asymmetry.max()], 
             [pre_asymmetry.min(), pre_asymmetry.max()], 
             'k--', lw=2, alpha=0.3, label='No change')
axes[0].set_xlabel('Pre-intervention Asymmetry', fontsize=12)
axes[0].set_ylabel('Post-intervention Asymmetry', fontsize=12)
axes[0].set_title('Frontal Alpha Asymmetry: Pre vs Post', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='gray', linestyle=':', alpha=0.5)
axes[0].axvline(x=0, color='gray', linestyle=':', alpha=0.5)

# Change distribution
axes[1].hist(diff, bins=15, color='purple', alpha=0.7, edgecolor='black')
axes[1].axvline(0, color='red', linestyle='--', linewidth=2, label='No change')
axes[1].axvline(diff.mean(), color='darkblue', linestyle='-', linewidth=2,
                label=f'Mean change: {diff.mean():.2f}')
axes[1].set_xlabel('Change in Asymmetry (Post - Pre)', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Distribution of Asymmetry Change', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 9. Practical Recommendations

### Data Collection
- **Minimum duration**: 2-3 minutes for stable estimates
- **Eye artifacts**: Use ICA or reject trials with blinks
- **Reference**: Average or mastoid reference
- **Baseline**: Collect resting baseline for comparison

### Analysis Choices
- **Frequency band**: Typical 8-13 Hz, but can individualize
- **Method**: Log ratio more robust to individual differences
- **Electrodes**: F3/F4 most common, can use F7/F8 or AF3/AF4

### Interpretation Caveats
- State vs. trait: Asymmetry can reflect stable trait or momentary state
- Individual differences: Large between-person variability
- Not emotion-specific: Indicates approach/withdrawal, not specific emotions
- Context-dependent: Interpretation depends on task and context

### Integration with Other Measures
- Correlate with PANAS for validation
- Combine with behavioral approach/avoidance tasks
- Track alongside music generation parameters
- Use for real-time neurofeedback

## References

Davidson, R. J. (1992). Anterior cerebral asymmetry and the nature of emotion. *Brain and Cognition*, 20(1), 125-151.

Coan, J. A., & Allen, J. J. (2004). Frontal EEG asymmetry as a moderator and mediator of emotion. *Biological Psychology*, 67(1-2), 7-50.

Harmon-Jones, E., Gable, P. A., & Peterson, C. K. (2010). The role of asymmetric frontal cortical activity in emotion-related phenomena: A review and update. *Biological Psychology*, 84(3), 451-462.